In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import sklearn as sk
from sklearn import decomposition as dec

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cpu device


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [5]:
model = NeuralNetwork().to(device)
print(model)


for name, params in model.named_parameters():
    print(name, params.shape)



NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
linear_relu_stack.0.weight torch.Size([512, 784])
linear_relu_stack.0.bias torch.Size([512])
linear_relu_stack.2.weight torch.Size([512, 512])
linear_relu_stack.2.bias torch.Size([512])
linear_relu_stack.4.weight torch.Size([10, 512])
linear_relu_stack.4.bias torch.Size([10])


In [6]:
X = torch.rand(1, 28, 28, device=device)
print('the tensor', X)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
print('pred probability_', pred_probab)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")


the tensor tensor([[[4.2696e-01, 1.4153e-01, 7.5251e-01, 2.9177e-01, 5.5462e-01,
          6.8289e-01, 3.7364e-01, 1.2884e-01, 3.9452e-01, 3.2259e-01,
          3.3647e-01, 6.3034e-02, 6.0368e-01, 2.2058e-03, 8.3479e-01,
          6.2475e-01, 3.9060e-01, 6.9036e-01, 5.8445e-01, 6.4414e-01,
          7.4710e-01, 7.4345e-01, 4.4199e-01, 4.3336e-01, 4.6245e-01,
          7.0071e-01, 8.5687e-01, 5.7807e-01],
         [2.7409e-01, 7.5895e-01, 4.1701e-01, 3.1125e-01, 6.6099e-01,
          5.0300e-01, 5.1150e-01, 4.0186e-01, 4.5110e-01, 4.3538e-02,
          9.6169e-01, 5.9079e-01, 2.7195e-01, 1.9811e-01, 8.1633e-01,
          8.7977e-01, 1.1034e-01, 7.4785e-01, 9.2868e-01, 4.3496e-01,
          5.5518e-01, 3.6045e-01, 8.8264e-01, 8.1049e-01, 3.6671e-02,
          7.6521e-02, 1.7983e-01, 4.4812e-01],
         [3.7915e-01, 5.6466e-01, 4.4387e-02, 2.4941e-01, 7.7652e-01,
          2.1149e-01, 3.1657e-01, 3.9956e-01, 1.7366e-01, 8.5122e-01,
          3.1438e-01, 3.9172e-01, 6.8974e-01, 3.0559e-0

In [7]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())


torch.Size([3, 28, 28])


In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())


torch.Size([3, 784])


In [9]:
new_image = torch.rand(3,28,28)
print(new_image.shape)
new_image_flat = nn.Flatten()(new_image)
print(new_image_flat.shape)

torch.Size([3, 28, 28])
torch.Size([3, 784])


In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(new_image_flat)
print(hidden1.size())
print(hidden1)

torch.Size([3, 20])
tensor([[-0.0431,  0.2953,  0.2094, -0.2261,  0.1049,  0.7387,  0.1223, -0.0431,
          0.0526,  0.0228,  0.2656, -0.1338,  0.1667, -0.1056,  0.5041,  0.3411,
         -0.3468, -0.0540,  0.3019, -0.0029],
        [-0.0167,  0.1232, -0.1887,  0.1269,  0.3108,  0.4942, -0.0067,  0.1927,
          0.1814,  0.2870, -0.0220, -0.0643,  0.2641, -0.3040,  0.4749,  0.2696,
         -0.1378,  0.0864,  0.2127,  0.1048],
        [ 0.1234,  0.5102,  0.3264,  0.2336,  0.1470,  0.7030, -0.0779, -0.2489,
          0.2600, -0.0125,  0.1482, -0.6005, -0.0117, -0.4549,  0.6568, -0.1463,
         -0.2320,  0.2022,  0.0350, -0.3296]], grad_fn=<AddmmBackward0>)


In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")


Before ReLU: tensor([[-0.0431,  0.2953,  0.2094, -0.2261,  0.1049,  0.7387,  0.1223, -0.0431,
          0.0526,  0.0228,  0.2656, -0.1338,  0.1667, -0.1056,  0.5041,  0.3411,
         -0.3468, -0.0540,  0.3019, -0.0029],
        [-0.0167,  0.1232, -0.1887,  0.1269,  0.3108,  0.4942, -0.0067,  0.1927,
          0.1814,  0.2870, -0.0220, -0.0643,  0.2641, -0.3040,  0.4749,  0.2696,
         -0.1378,  0.0864,  0.2127,  0.1048],
        [ 0.1234,  0.5102,  0.3264,  0.2336,  0.1470,  0.7030, -0.0779, -0.2489,
          0.2600, -0.0125,  0.1482, -0.6005, -0.0117, -0.4549,  0.6568, -0.1463,
         -0.2320,  0.2022,  0.0350, -0.3296]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.2953, 0.2094, 0.0000, 0.1049, 0.7387, 0.1223, 0.0000, 0.0526,
         0.0228, 0.2656, 0.0000, 0.1667, 0.0000, 0.5041, 0.3411, 0.0000, 0.0000,
         0.3019, 0.0000],
        [0.0000, 0.1232, 0.0000, 0.1269, 0.3108, 0.4942, 0.0000, 0.1927, 0.1814,
         0.2870, 0.0000, 0.0000, 0.2641, 0.0000, 0.47

In [24]:
seq_modules = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 512),
    nn.ReLU(),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
input_image2 = torch.rand(3, 28, 28)
logits = seq_modules(input_image2)

print('shape', logits.shape)


print('tensor output: ', logits)


softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

print(pred_probab.shape)

shape torch.Size([3, 10])
tensor output:  tensor([[ 0.0310,  0.0588, -0.0087,  0.0181,  0.0285,  0.0176,  0.0396,  0.0557,
         -0.0008, -0.0584],
        [ 0.0332,  0.1023,  0.0550,  0.0579,  0.0206, -0.0187,  0.0371,  0.0556,
         -0.0295, -0.0864],
        [ 0.0318,  0.0441,  0.0232,  0.0444,  0.0319, -0.0053,  0.0179,  0.0415,
          0.0287, -0.0619]], grad_fn=<AddmmBackward0>)
torch.Size([3, 10])


In [25]:
print(f"Model structure: {seq_modules}\n\n")

for name, param in seq_modules.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")


Model structure: Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=256, bias=True)
  (4): ReLU()
  (5): Linear(in_features=256, out_features=10, bias=True)
)


Layer: 1.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0307,  0.0080,  0.0082,  ...,  0.0229,  0.0047, -0.0140],
        [ 0.0166,  0.0079,  0.0144,  ...,  0.0328,  0.0299,  0.0040]],
       grad_fn=<SliceBackward0>) 

Layer: 1.bias | Size: torch.Size([512]) | Values : tensor([ 0.0039, -0.0316], grad_fn=<SliceBackward0>) 

Layer: 3.weight | Size: torch.Size([256, 512]) | Values : tensor([[ 0.0220,  0.0403, -0.0094,  ...,  0.0276,  0.0143, -0.0341],
        [ 0.0140,  0.0297,  0.0095,  ...,  0.0387,  0.0167,  0.0424]],
       grad_fn=<SliceBackward0>) 

Layer: 3.bias | Size: torch.Size([256]) | Values : tensor([-0.0180, -0.0154], grad_fn=<SliceBackward0>) 

Layer: 5.weight | Size: torch.Size([10, 

In [26]:
for name, params in seq_modules.named_parameters():
    print(name,'--', params.shape)

1.weight -- torch.Size([512, 784])
1.bias -- torch.Size([512])
3.weight -- torch.Size([256, 512])
3.bias -- torch.Size([256])
5.weight -- torch.Size([10, 256])
5.bias -- torch.Size([10])


In [28]:
print('^^^^')
print(seq_modules[3].weight.shape)

^^^^
torch.Size([256, 512])


In [29]:
weight_param = {}

for idx, (name, params) in enumerate(seq_modules.named_parameters()):
    if 'weight' in name:
        
        if idx == 0:
           continue
        print(name)
        print(params.shape)
        weight_param[name] = params
        print('dic', weight_param[name].shape )




3.weight
torch.Size([256, 512])
dic torch.Size([256, 512])
5.weight
torch.Size([10, 256])
dic torch.Size([10, 256])


In [30]:


for key,value in weight_param.items():

    
    x = weight_param[key].detach().numpy()
    print('before reshaping:', x.shape)

    arb = 100   
    if x.shape[0] < x.shape[1] and x.shape[0]<=arb:
     dim = int(x.shape[1]/2)
     x = x.reshape((-1 ,dim))
    x = x.T
    print('after reshaping:',x.shape)
    min_com = min(x.shape[0], x.shape[1])
    small_pca = dec.PCA(n_components=int(min_com/2))
    small_pca.fit(x)
    dr_small  = small_pca.transform(x)
    print('dmiensional reduction PCA', dr_small.shape)
    dr_small = dr_small.T
    print('Transpose', dr_small.shape)

    i = key.split('.')
    i = int(i[0])


    seq_modules[i].weight = nn.Parameter(torch.tensor(dr_small))
    # seq_modules[i] = nn.Linear(in_features=dr_small.shape[1], 
    #                    out_features=dr_small.shape[0], bias=True)

    print('Assigned value to network weight',seq_modules[i].weight.shape)
    print('Module', seq_modules[i])
    print('----------------------------------------')




before reshaping: (256, 512)
after reshaping: (512, 256)
dmiensional reduction PCA (512, 128)
Transpose (128, 512)
Assigned value to network weight torch.Size([128, 512])
Module Linear(in_features=512, out_features=256, bias=True)
----------------------------------------
before reshaping: (10, 256)
after reshaping: (128, 20)
dmiensional reduction PCA (128, 10)
Transpose (10, 128)
Assigned value to network weight torch.Size([10, 128])
Module Linear(in_features=256, out_features=10, bias=True)
----------------------------------------


In [36]:
for name, param in seq_modules.named_parameters():
    print(name, param.shape)
    

1.weight torch.Size([512, 784])
1.bias torch.Size([512])
3.weight torch.Size([128, 512])
3.bias torch.Size([256])
5.weight torch.Size([10, 128])
5.bias torch.Size([10])


In [35]:
print(seq_modules[3].bias.shape)


torch.Size([256])


In [37]:
print(seq_modules.eval())
pca = dec.PCA(n_components=2)
temp = seq_modules[3].bias.detach().numpy()
temp = temp.reshape(64,-1)
print(temp.shape)
pca.fit(temp)
new_bias = pca.transform(temp)
new_bias = new_bias.reshape((128))
seq_modules[3].bias = nn.Parameter(torch.tensor(new_bias))
print(seq_modules[3].bias.shape)

result = seq_modules(input_image2)
print(result.shape)

print(result)


Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=256, bias=True)
  (4): ReLU()
  (5): Linear(in_features=256, out_features=10, bias=True)
)
(64, 4)
torch.Size([128])
torch.Size([3, 10])
tensor([[-0.0134,  0.0868,  0.0486,  0.0409, -0.0349,  0.1044,  0.0408, -0.0313,
          0.0672, -0.0822],
        [ 0.0342,  0.0885,  0.0799,  0.0814, -0.0293,  0.0430,  0.0711, -0.0375,
          0.0579, -0.0517],
        [ 0.0633,  0.1137,  0.0433,  0.0168, -0.0337,  0.0327,  0.0352, -0.0210,
          0.0520, -0.0577]], grad_fn=<AddmmBackward0>)
